# OSale Dataset Generation

On this notebook we will proceed to generate a dataset based on the Brazilian ecommerce public dataset. The dataset consist on a flat file that contains the variables that will be used for modeling. 

The steps for the generation of the dataser are:
 
1. Define the type of prediction we want to model. 
2. Define the predicted variable (outcome). 
3. Define the time periods for the data. The time cutoff will be used for defining the independent variables (based on the time before the time cutoff), and the outcome (based on the time after the time cutoff).
4. Add the independent variables to the table.

The goal of our model will be to predict which sellers will stay on the marked for at least 3 month after their first sell.


In [55]:
if(!require(DBI)){install.packages("DBI"); require(DBI)}
if(!require(RSQLite)){install.packages("RSQLite"); require(RSQLite)}
require(dplyr)

In [56]:
ROOT_PATH = "/home/tomas/Documents/Technion/class_project/"
DATA_PATH = "/home/tomas/Documents/Technion/class_project/data/raw/"
DB_FILE = paste0(DATA_PATH,"/olist.db") 

In [57]:
##############################################################################
##########                DATABASE FUNCTIONS                     #############
##############################################################################

#### List the name of the tables on the database 
listTables <- function(db=DB_FILE) {
    require(DBI)
    require(RSQLite)
    con <- dbConnect(SQLite(), DB_FILE)
    ### list the tables on the DB
    res <- dbListTables(con)
    return(res)
}
            
#### Read function to import data from the SQL to a pandas dataframe.
readSQL <- function(query, db=DB_FILE) {
    require(DBI)
    require(RSQLite)
    con <- dbConnect(SQLite(), DB_FILE)
    df <- dbGetQuery(con, query)
    return(df)
}

#### Write a pandas dataframe into an SQL table. Use overwrite=True if you want to delete 
#### first a pre-existent table with the same name. Use append=True if you want to append
#### the data in the dataframe to a pre-existent table.
writeSQL <- function(df,tablename,overwrite=FALSE, append=FALSE,db=DB_FILE) {
    require(DBI)
    require(RSQLite)
    con <- dbConnect(SQLite(), DB_FILE)
    dbWriteTable(con,tablename,df,overwrite,append)
}

#### Generate a view based on the provided SQL query
createView <- function(viewname, query) {
    require(DBI)
    require(RSQLite)
    con <- dbConnect(SQLite(), DB_FILE)
    view = paste0('CREATE VIEW ',viewname,' AS \n', query)
    dbSendQuery(con, view)
}

In [58]:
listTables()

[1] "closed_deals"                      "cust_sell_geo_v"                  
 [3] "customer_geolocation_v"            "customer_sellers_v"               
 [5] "customers"                         "geolocation"                      
 [7] "marketing_qualified_leads"         "order_items"                      
 [9] "order_payment_final_v"             "order_payments"                   
[11] "order_product_counts_v"            "order_reviews"                    
[13] "orders"                            "product_category_name_translation"
[15] "product_cost_dim"                  "product_costs_v"                  
[17] "products"                          "products_by_sellers_v"            
[19] "seller_geolocation_v"              "seller_prodcat_v"                 
[21] "seller_times"                      "sellers"                          
[23] "sellers_dataset"

In [5]:
sellers = readSQL("SELECT * FROM sellers")

In [6]:
dim(sellers)

[1] 3095    4

In [7]:
head(sellers)

seller_id,seller_zip_code_prefix,seller_city,seller_state
3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP
c240c4061717ac1806ae6ee72be3533b,20920,rio de janeiro,RJ


As we can see, we have 3095 sellers. we need now to check when they entered Olist. For thise we check when the first order was made.

In [8]:
order_items = readSQL("SELECT * FROM order_items")
head(order_items)

order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14
00048cc3ae777c65dbb7d2a0634bc1ea,1,ef92defde845ab8450f9d70c526ef70f,6426d21aca402a131fc0a5d0960a3c90,2017-05-23 03:55:27,21.90,12.69


In [9]:
query <- "
SELECT a.seller_id, b.order_purchase_timestamp AS transaction_date, COUNT(1) AS transactions_count
FROM order_items a
INNER JOIN orders b
    ON a.order_id = b.order_id
GROUP BY a.seller_id, b.order_purchase_timestamp
"

seller_transactions = readSQL(query)
dim(seller_transactions)
head(seller_transactions)

[1] 99891     3

seller_id,transaction_date,transactions_count
0015a82c2db000af6aaaf3ae2ecb0532,2017-09-26 22:17:05,1
0015a82c2db000af6aaaf3ae2ecb0532,2017-10-12 13:33:22,1
0015a82c2db000af6aaaf3ae2ecb0532,2017-10-18 08:16:34,1
001cca7ae9ae17fb1caed9dfb1094831,2017-02-04 19:06:04,1
001cca7ae9ae17fb1caed9dfb1094831,2017-02-18 23:26:24,5
001cca7ae9ae17fb1caed9dfb1094831,2017-02-23 11:42:38,1


In [34]:
seller_times <- seller_transactions %>% 
    group_by(seller_id) %>% 
    summarise(first_date=min(as.Date(transaction_date)), last_date=max(as.Date(transaction_date)),
             time_in_market=max(as.Date(transaction_date)) - min(as.Date(transaction_date)))
dim(seller_times)
head(seller_times)

## write table to sqlite for later use!
writeSQL(seller_times,"seller_times")

[1] 3095    4

seller_id,first_date,last_date,time_in_market
0015a82c2db000af6aaaf3ae2ecb0532,2017-09-26,2017-10-18,22 days
001cca7ae9ae17fb1caed9dfb1094831,2017-02-04,2018-07-12,523 days
001e6ad469a905060d959994f1b41e4f,2018-08-07,2018-08-07,0 days
002100f778ceb8431b7a1020ff7ab48f,2017-09-14,2018-04-12,210 days
003554e2dce176b5555353e4f3555ac8,2017-12-15,2017-12-15,0 days
004c9cd9d87a3c30c522c48c4fc07416,2017-01-27,2018-05-03,461 days


In [11]:
## We need to get at least 3 month of follow-up data to define our outcome !!!
max(seller_times$last_date)
seller_times$time_to_end <- as.Date('2018-06-01') - seller_times$last_date 
head(seller_times)

[1] "2018-09-03"

seller_id,first_date,last_date,time_in_market,time_to_end
0015a82c2db000af6aaaf3ae2ecb0532,2017-09-26,2017-10-18,22 days,226 days
001cca7ae9ae17fb1caed9dfb1094831,2017-02-04,2018-07-12,523 days,-41 days
001e6ad469a905060d959994f1b41e4f,2018-08-07,2018-08-07,0 days,-67 days
002100f778ceb8431b7a1020ff7ab48f,2017-09-14,2018-04-12,210 days,50 days
003554e2dce176b5555353e4f3555ac8,2017-12-15,2017-12-15,0 days,168 days
004c9cd9d87a3c30c522c48c4fc07416,2017-01-27,2018-05-03,461 days,29 days


In [12]:
nrow(seller_times %>% filter(time_to_end >= 90))
nrow(seller_times %>% filter(time_in_market >= 90 & time_to_end >= 90))


[1] 903

[1] 332

As we see, there are only 903 sellers that entered at least three months before the maximum follow up period. Of them, 332 sellers were on market for more than 90 days (3 months). Based on this we will add to the seller dataframe the outcome:

In [23]:
seller_times  <- seller_times %>% filter(time_to_end >= 90) %>% mutate(outcome=ifelse(time_in_market >= 90,1,0))
head(seller_times)
nrow(seller_times)

seller_id,first_date,last_date,time_in_market,time_to_end,outcome
0015a82c2db000af6aaaf3ae2ecb0532,2017-09-26,2017-10-18,22 days,226 days,0
003554e2dce176b5555353e4f3555ac8,2017-12-15,2017-12-15,0 days,168 days,0
00ab3eff1b5192e5f1a63bcecfee11c8,2017-03-18,2017-03-18,0 days,440 days,0
010543a62bd80aa422851e79a3bc7540,2017-07-04,2017-08-02,29 days,303 days,0
011b0eaba87386a2ae96a7d32bb531d1,2016-10-06,2016-10-06,0 days,603 days,0
01bcc9d254a0143f0ce9791b960b2a47,2017-08-12,2018-02-02,174 days,119 days,1


[1] 903

Now, we will join the sellers dataframe with the seller_times, which has the outcome variable. As we don't need the dates and the time in market variables on our flat file, we ignore them. 

In [24]:
sellers <- inner_join(seller_times[,c("seller_id","outcome")], sellers)
head(sellers)
nrow(sellers)

Joining, by = c("seller_id", "outcome")


seller_id,outcome,seller_zip_code_prefix,seller_city,seller_state,categories_count,products_count,cat_1,cat_2,cat_3,⋯,cat_65,cat_66,cat_67,cat_68,cat_69,cat_70,cat_71,cat_72,cat_73,cat_74
0015a82c2db000af6aaaf3ae2ecb0532,0,9080,santo andre,SP,1,1,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
003554e2dce176b5555353e4f3555ac8,0,74565,goiania,GO,1,1,1,0,0,⋯,0,0,0,0,0,0,0,0,0,0
00ab3eff1b5192e5f1a63bcecfee11c8,0,4164,sao paulo,SP,1,1,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
010543a62bd80aa422851e79a3bc7540,0,1212,sao paulo,SP,1,1,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
011b0eaba87386a2ae96a7d32bb531d1,0,17580,pompeia,SP,1,1,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
01bcc9d254a0143f0ce9791b960b2a47,1,76400,uruacu,GO,1,3,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


[1] 903

The next step is to add the variables that we think will help us in our modeling work. The variables shoul not indicate future behaviours. Only variables that were known at the time of the first time entering Olist. However, as we don't know when the variables were introduced and for the purpose of this exercise, we will peek some of them that has the highest probability to be known at entering time. 
 
On real projects, our customer have this data and we have to be sure we have access to it. 
 
We will begin with the products. We will add the categories that the seller is selling. We need first to know how many categories there are, the maximum number of categories the sellers are offering, etc.

In [35]:
query <- "
SELECT a.seller_id, b.product_category_name, COUNT(DISTINCT a.product_id) products_count
FROM order_items a
INNER JOIN products b
    ON a.product_id = b.product_id
GROUP BY a.seller_id, b.product_category_name
"

seller_prodcat <- readSQL(query)
dim(seller_prodcat)
head(seller_prodcat)

# write view on sqlite for later use
createView("seller_prodcat_v",query)

[1] 6616    3

seller_id,product_category_name,products_count
0015a82c2db000af6aaaf3ae2ecb0532,eletroportateis,1
001cca7ae9ae17fb1caed9dfb1094831,construcao_ferramentas_construcao,5
001cca7ae9ae17fb1caed9dfb1094831,ferramentas_jardim,6
001e6ad469a905060d959994f1b41e4f,esporte_lazer,1
002100f778ceb8431b7a1020ff7ab48f,moveis_decoracao,24
003554e2dce176b5555353e4f3555ac8,,1


<SQLiteResult>
  SQL  CREATE VIEW seller_prodcat_v AS 

SELECT a.seller_id, b.product_category_name, COUNT(DISTINCT a.product_id) products_count
FROM order_items a
INNER JOIN products b
    ON a.product_id = b.product_id
GROUP BY a.seller_id, b.product_category_name

  ROWS Fetched: 0 [complete]
       Changed: 0

In [16]:
### count products categories
seller_prodcat %>% group_by(seller_id) %>% summarise(categ_count=n()) %>% group_by(categ_count) %>% tally()

categ_count,n
1,1689
2,690
3,285
4,149
5,94
6,55
7,39
8,27
9,17
10,9


Most of the sellers offer only one category, but there are many that offer more than one. We can use the number of categories as one of our variables. We can also add the number of products offered. Additionally, we can add many categories as individual variables. Those variables will be binary variables with 0/1 as their values.

In [25]:
## add the number of products and categories to the dataset
sellers <- left_join(sellers, 
    (seller_prodcat %>% group_by(seller_id) %>% 
     summarise(categories_count=n(), products_count=sum(products_count)))
)
head(sellers)
nrow(sellers)

Joining, by = c("seller_id", "categories_count", "products_count")


seller_id,outcome,seller_zip_code_prefix,seller_city,seller_state,categories_count,products_count,cat_1,cat_2,cat_3,⋯,cat_65,cat_66,cat_67,cat_68,cat_69,cat_70,cat_71,cat_72,cat_73,cat_74
0015a82c2db000af6aaaf3ae2ecb0532,0,9080,santo andre,SP,1,1,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
003554e2dce176b5555353e4f3555ac8,0,74565,goiania,GO,1,1,1,0,0,⋯,0,0,0,0,0,0,0,0,0,0
00ab3eff1b5192e5f1a63bcecfee11c8,0,4164,sao paulo,SP,1,1,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
010543a62bd80aa422851e79a3bc7540,0,1212,sao paulo,SP,1,1,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
011b0eaba87386a2ae96a7d32bb531d1,0,17580,pompeia,SP,1,1,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
01bcc9d254a0143f0ce9791b960b2a47,1,76400,uruacu,GO,1,3,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


[1] 903

In [26]:
## add each category as a variable
cats <- readSQL("SELECT DISTINCT product_category_name FROM products")
#cats <- as.character(cats)
nn <- 1:nrow(cats)
cats <- data.frame(category_id=nn,category_desc=cats)
cats

category_id,product_category_name
1,perfumaria
2,artes
3,esporte_lazer
4,bebes
5,utilidades_domesticas
6,instrumentos_musicais
7,cool_stuff
8,moveis_decoracao
9,eletrodomesticos
10,brinquedos


In [27]:
## one-hot-encoding...
one_hot_encoding <- function(data,id,category,prefix) {
    require(dplyr)
    cat <- levels(factor(data[[category]]))
    res <- data %>% group_by_(id) %>% select_(id) %>% distinct_()
    for (l in 1:length(cat)) {
        #cname <- paste(prefix,l,sep="_")
        #data[[cname]] <- ifelse(data[[category]]==cat[l],1,0)
        #cc1 <- data %>% group_by_(id) %>% summarise_(cname=max(cname)) %>% select_(id,cname)
        #res <- cbind(res, cc1[,2])
        cname <- paste("cat",l,sep="_")
        ncat <- "newcat"
        data[[ncat]] <- ifelse(data[[category]]==cat[l],1,0)
        cc1 <- data %>% select_(id,ncat) %>% group_by_(id) %>% 
               summarise(newcat=max(newcat))
        res[[cname]] <- cc1$newcat
    }
    return(res)
}


In [28]:
head(one_hot_encoding(data=seller_prodcat,id="seller_id",category="product_category_name",prefix="categ"))

dim(one_hot_encoding(seller_prodcat,"seller_id","product_category_name", "categ"))

seller_id,cat_1,cat_2,cat_3,cat_4,cat_5,cat_6,cat_7,cat_8,cat_9,⋯,cat_65,cat_66,cat_67,cat_68,cat_69,cat_70,cat_71,cat_72,cat_73,cat_74
0015a82c2db000af6aaaf3ae2ecb0532,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
001cca7ae9ae17fb1caed9dfb1094831,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
001e6ad469a905060d959994f1b41e4f,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
002100f778ceb8431b7a1020ff7ab48f,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
003554e2dce176b5555353e4f3555ac8,1,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
004c9cd9d87a3c30c522c48c4fc07416,1,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


[1] 3095   75

In [21]:
sellers <- inner_join(sellers,one_hot_encoding(seller_prodcat,"seller_id","product_category_name", "categ"))

Joining, by = "seller_id"


In [29]:
head(sellers)
nrow(sellers)

seller_id,outcome,seller_zip_code_prefix,seller_city,seller_state,categories_count,products_count,cat_1,cat_2,cat_3,⋯,cat_65,cat_66,cat_67,cat_68,cat_69,cat_70,cat_71,cat_72,cat_73,cat_74
0015a82c2db000af6aaaf3ae2ecb0532,0,9080,santo andre,SP,1,1,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
003554e2dce176b5555353e4f3555ac8,0,74565,goiania,GO,1,1,1,0,0,⋯,0,0,0,0,0,0,0,0,0,0
00ab3eff1b5192e5f1a63bcecfee11c8,0,4164,sao paulo,SP,1,1,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
010543a62bd80aa422851e79a3bc7540,0,1212,sao paulo,SP,1,1,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
011b0eaba87386a2ae96a7d32bb531d1,0,17580,pompeia,SP,1,1,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
01bcc9d254a0143f0ce9791b960b2a47,1,76400,uruacu,GO,1,3,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


[1] 903

In [41]:
### Number of competitors @ entry date

query <- "
SELECT s1.seller_id, COUNT(DISTINCT s2.seller_id) competitors_count
FROM seller_prodcat_v s1
INNER JOIN seller_times t1
    ON s1.seller_id = t1.seller_id
LEFT OUTER JOIN seller_prodcat_v s2
    ON s1.product_category_name = s2.product_category_name
    AND s1.seller_id <> s2.seller_id
INNER JOIN seller_times t2
    ON s2.seller_id = t2.seller_id
WHERE t2.first_date <= t1.first_date
GROUP BY s1.seller_id
"
head(readSQL(query))
nrow(readSQL(query))

seller_id,competitors_count
0015a82c2db000af6aaaf3ae2ecb0532,56
001cca7ae9ae17fb1caed9dfb1094831,57
001e6ad469a905060d959994f1b41e4f,471
002100f778ceb8431b7a1020ff7ab48f,236
003554e2dce176b5555353e4f3555ac8,204
004c9cd9d87a3c30c522c48c4fc07416,59


[1] 3092

In [43]:
sellers <- left_join(sellers, readSQL(query))
head(sellers)

Joining, by = "seller_id"


seller_id,outcome,seller_zip_code_prefix,seller_city,seller_state,categories_count,products_count,cat_1,cat_2,cat_3,⋯,cat_66,cat_67,cat_68,cat_69,cat_70,cat_71,cat_72,cat_73,cat_74,competitors_count
0015a82c2db000af6aaaf3ae2ecb0532,0,9080,santo andre,SP,1,1,0,0,0,⋯,0,0,0,0,0,0,0,0,0,56
003554e2dce176b5555353e4f3555ac8,0,74565,goiania,GO,1,1,1,0,0,⋯,0,0,0,0,0,0,0,0,0,204
00ab3eff1b5192e5f1a63bcecfee11c8,0,4164,sao paulo,SP,1,1,0,0,0,⋯,0,0,0,0,0,0,0,0,0,106
010543a62bd80aa422851e79a3bc7540,0,1212,sao paulo,SP,1,1,0,0,0,⋯,0,0,0,0,0,0,0,0,0,124
011b0eaba87386a2ae96a7d32bb531d1,0,17580,pompeia,SP,1,1,0,0,0,⋯,0,0,0,0,0,0,0,0,0,17
01bcc9d254a0143f0ce9791b960b2a47,1,76400,uruacu,GO,1,3,0,0,0,⋯,0,0,0,0,0,0,0,0,0,183


In [44]:
writeSQL(sellers, "sellers_dataset")

In [49]:
head(readSQL("SELECT * FROM sellers_dataset"))
dim(readSQL("SELECT * FROM sellers_dataset"))


seller_id,outcome,seller_zip_code_prefix,seller_city,seller_state,categories_count,products_count,cat_1,cat_2,cat_3,⋯,cat_66,cat_67,cat_68,cat_69,cat_70,cat_71,cat_72,cat_73,cat_74,competitors_count
0015a82c2db000af6aaaf3ae2ecb0532,0,9080,santo andre,SP,1,1,0,0,0,⋯,0,0,0,0,0,0,0,0,0,56
003554e2dce176b5555353e4f3555ac8,0,74565,goiania,GO,1,1,1,0,0,⋯,0,0,0,0,0,0,0,0,0,204
00ab3eff1b5192e5f1a63bcecfee11c8,0,4164,sao paulo,SP,1,1,0,0,0,⋯,0,0,0,0,0,0,0,0,0,106
010543a62bd80aa422851e79a3bc7540,0,1212,sao paulo,SP,1,1,0,0,0,⋯,0,0,0,0,0,0,0,0,0,124
011b0eaba87386a2ae96a7d32bb531d1,0,17580,pompeia,SP,1,1,0,0,0,⋯,0,0,0,0,0,0,0,0,0,17
01bcc9d254a0143f0ce9791b960b2a47,1,76400,uruacu,GO,1,3,0,0,0,⋯,0,0,0,0,0,0,0,0,0,183


[1] 901  82

# Exploratory analysis and data cleansing
 
 - Identify data types and data distributions
 - Identify quantity and type of missing values
 - Identify outliers (univariate and multivariate)
 - Identify data patterns (using cluster analysis / PCA)


In [53]:
library(mechkar)

In [67]:
#seller <- readSQL("SELECT * FROM seller_dataset")
head(sellers)
dim(sellers)
sellers$outcome <- factor(sellers$outcome)

seller_id,outcome,seller_zip_code_prefix,seller_city,seller_state,categories_count,products_count,cat_1,cat_2,cat_3,⋯,cat_66,cat_67,cat_68,cat_69,cat_70,cat_71,cat_72,cat_73,cat_74,competitors_count
0015a82c2db000af6aaaf3ae2ecb0532,0,9080,santo andre,SP,1,1,0,0,0,⋯,0,0,0,0,0,0,0,0,0,56
003554e2dce176b5555353e4f3555ac8,0,74565,goiania,GO,1,1,1,0,0,⋯,0,0,0,0,0,0,0,0,0,204
00ab3eff1b5192e5f1a63bcecfee11c8,0,4164,sao paulo,SP,1,1,0,0,0,⋯,0,0,0,0,0,0,0,0,0,106
010543a62bd80aa422851e79a3bc7540,0,1212,sao paulo,SP,1,1,0,0,0,⋯,0,0,0,0,0,0,0,0,0,124
011b0eaba87386a2ae96a7d32bb531d1,0,17580,pompeia,SP,1,1,0,0,0,⋯,0,0,0,0,0,0,0,0,0,17
01bcc9d254a0143f0ce9791b960b2a47,1,76400,uruacu,GO,1,3,0,0,0,⋯,0,0,0,0,0,0,0,0,0,183


[1] 901  82

In [70]:
summary(sellers$outcome)

0   1 
569 332

In [96]:
excludeUniqueValue <- function(data) {
    nm <- names(data)
    res <- NULL
    for(n in nm) {
        if(length(unique(data[[n]]))==1) {
            data[[n]] <- NULL
            res <- c(res, n)
            #print(n)
        }
    }
    if(length(res) > 0) {
        message("The following variables had only one unique values and were removed")
        message(res)
    }
    return(data)
}

In [97]:
summary(excludeUniqueValue(sellers))

The following variables had only one unique values and were removed
cat_7cat_17cat_19cat_25cat_43cat_48cat_62cat_67cat_69


  seller_id         outcome seller_zip_code_prefix seller_city       
 Length:901         0:569   Min.   : 1026          Length:901        
 Class :character   1:332   1st Qu.: 6787          Class :character  
 Mode  :character           Median :15170          Mode  :character  
                            Mean   :33129                            
                            3rd Qu.:74210                            
                            Max.   :99700                            
 seller_state       categories_count products_count       cat_1       
 Length:901         Min.   : 1.00    Min.   : 1.000   Min.   :0.0000  
 Class :character   1st Qu.: 1.00    1st Qu.: 1.000   1st Qu.:0.0000  
 Mode  :character   Median : 1.00    Median : 2.000   Median :0.0000  
                    Mean   : 1.58    Mean   : 3.788   Mean   :0.1743  
                    3rd Qu.: 2.00    3rd Qu.: 4.000   3rd Qu.:0.0000  
                    Max.   :11.00    Max.   :53.000   Max.   :1.0000  
     cat_2   

In [98]:
sellers <- excludeUniqueValue(sellers)
head(sellers)

The following variables had only one unique values and were removed
cat_7cat_17cat_19cat_25cat_43cat_48cat_62cat_67cat_69


seller_id,outcome,seller_zip_code_prefix,seller_city,seller_state,categories_count,products_count,cat_1,cat_2,cat_3,⋯,cat_64,cat_65,cat_66,cat_68,cat_70,cat_71,cat_72,cat_73,cat_74,competitors_count
0015a82c2db000af6aaaf3ae2ecb0532,0,9080,santo andre,SP,1,1,0,0,0,⋯,0,0,0,0,0,0,0,0,0,56
003554e2dce176b5555353e4f3555ac8,0,74565,goiania,GO,1,1,1,0,0,⋯,0,0,0,0,0,0,0,0,0,204
00ab3eff1b5192e5f1a63bcecfee11c8,0,4164,sao paulo,SP,1,1,0,0,0,⋯,0,0,0,0,0,0,0,0,0,106
010543a62bd80aa422851e79a3bc7540,0,1212,sao paulo,SP,1,1,0,0,0,⋯,0,0,0,0,0,0,0,0,0,124
011b0eaba87386a2ae96a7d32bb531d1,0,17580,pompeia,SP,1,1,0,0,0,⋯,0,0,0,0,0,0,0,0,0,17
01bcc9d254a0143f0ce9791b960b2a47,1,76400,uruacu,GO,1,3,0,0,0,⋯,0,0,0,0,0,0,0,0,0,183


In [109]:
#exploreData(data=sellers[,2:ncol(sellers)], y="outcome",factorSize = TRUE)
exploreData(data=sellers, factorSize = TRUE)

ERROR: Error in sort.list(y): 'x' must be atomic for 'sort.list'
Have you called 'sort' on a list?


In [105]:
ncol(sellers)

[1] 73